In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

Raw_Edited_v1 = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v1_combined_data.csv"  #combined_4x_raw_dataset_path
Raw_Edited_v2 = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v2_Sorted_Database.csv" #sorted_combined_dataset_path
Raw_Edited_v3 =  "drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v3_Sorted_Raw_Db_with_RUL.csv" #sorted combined with only chassis numbers present in repairs and RUL info

Repairs = "drive/My Drive/Thesis/data/PAC_REPAIRS_TRB_20190625_anonymized.csv" #repairs_dataset_path
Repairs_Edited_v1 = "drive/My Drive/Thesis/data/Data_Engineering/Edited_Repair_Data/v1_Sorted_Repairs.csv" #sorted_repairs_dataset_path




In [4]:
repair_data = pd.read_csv(Repairs_Edited_v1, low_memory=False)

In [5]:
repair_data['RepairDate']

0        2019-06-08 00:00:00.000000
1        2019-02-27 00:00:00.000000
2        2018-11-05 00:00:00.000000
3        2019-06-15 00:00:00.000000
4        2019-01-09 00:00:00.000000
                    ...            
13087    2018-02-10 00:00:00.000000
13088    2017-10-10 00:00:00.000000
13089    2017-10-10 00:00:00.000000
13090    2017-06-03 00:00:00.000000
13091    2017-06-03 00:00:00.000000
Name: RepairDate, Length: 13092, dtype: object

In [ ]:
sorted_data = pd.read_csv(Raw_Edited_v2, low_memory=False)

In [ ]:
sorted_data.head()


,BIWEEK_SEND_DATE,T_CHASSIS,INT_BWSD,LX_PMU_P1FH1_SLOPE_X_INDEX_10,LX_PMU_P1FH1_SLOPE_X_INDEX_11,LX_PMU_P1FH1_SLOPE_X_INDEX_12,LX_PMU_P1FH1_SLOPE_X_INDEX_13,LX_PMU_P1FH1_SLOPE_X_INDEX_14,LX_PMU_P1FH1_SLOPE_X_INDEX_15,LX_PMU_P1FH1_SLOPE_X_INDEX_16,LX_PMU_P1FH1_SLOPE_X_INDEX_17,LX_PMU_P1FH1_SLOPE_X_INDEX_18,LX_PMU_P1FH1_SLOPE_X_INDEX_19,LX_PMU_P1FH1_SLOPE_X_INDEX_2,LX_PMU_P1FH1_SLOPE_X_INDEX_20,LX_PMU_P1FH1_SLOPE_X_INDEX_21,LX_PMU_P1FH1_SLOPE_X_INDEX_22,LX_PMU_P1FH1_SLOPE_X_INDEX_23,LX_PMU_P1FH1_SLOPE_X_INDEX_24,LX_PMU_P1FH1_SLOPE_X_INDEX_25,LX_PMU_P1FH1_SLOPE_X_INDEX_26,LX_PMU_P1FH1_SLOPE_X_INDEX_27,LX_PMU_P1FH1_SLOPE_X_INDEX_28,LX_PMU_P1FH1_SLOPE_X_INDEX_29,LX_PMU_P1FH1_SLOPE_X_INDEX_3,LX_PMU_P1FH1_SLOPE_X_INDEX_30,LX_PMU_P1FH1_SLOPE_X_INDEX_31,LX_PMU_P1FH1_SLOPE_X_INDEX_4,LX_PMU_P1FH1_SLOPE_X_INDEX_5,LX_PMU_P1FH1_SLOPE_X_INDEX_6,LX_PMU_P1FH1_SLOPE_X_INDEX_7,LX_PMU_P1FH1_SLOPE_X_INDEX_8,LX_PMU_P1FH1_SLOPE_X_INDEX_9,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_1,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_10,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_11,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_12,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_13,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_14,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_15,...,VAR_U4X_BRAKE_DRIVE_REAR,VAR_FVX_FRONT_AXLE_LOAD,VAR_VWX_COMPRESSOR_CAPACITY,VAR_DAX_AXLE_ARRANGEMENT,VAR_H1B_TURBO_APPLICATION,VAR_FDX_REAR_AXLE_ARRANGEMENT,VAR_7PX_HYDRAULIC_PUMP_GEARBOX_MOUNTED,VAR_7NA_FRONT_SUSPENSION_INSTALLATION,VAR_LKX_DRIVER_STATE_SENSING_SYSTEM,VAR_QWX_TURBO,VAR_FZX_FRONT_AXLE_ARRANGEMENT,VAR_V6X_RETARDER,VAR_K4C_VEHICLE_UPTIME_SETTINGS,VAR_0RA_REGENERATION_CONTROLS_DPF,VAR_YKX_FRONT_SUSPENSION_SYSTEM,VAR_T3X_PTO_ENGINE_REAR_MOUNTED,VAR_PJX_ALTERNATOR_CAPACITY,VAR_41X_PARKING_HEATER,VAR_D7C_WHOLE_VEHICLE_TYPE_APPROVAL,VAR_5JB_TELEMATICS_COMMUNICAT_GATEWAYS,VAR_RWX_TRANSMISSION,VAR_DHX_ROAD_CONDITION,VAR_2EX_CAB_TILT_PUMP,VAR_D9C_EMISSION_LEVEL,VAR_DDX_CHASSIS_TYPE,VAR_O8A_ENGINE_PTO_TORQUE_CAPACITY,VAR_33A_CAB_HEIGHT,VAR_J2A_ACTIVE_STEERING,VAR_N9C_ADR_CLASSIFICATION,VFE_0001_HORSE_POWER,VFE_0002_GROSS_COMBINATION_WEIGHT_BIN,VFE_0003_ENGINE_GENERATION,VFE_0004_REAR_AXLE_LOAD_BIN,VFE_0005_REAR_AXLE_RATIO_BIN,VFE_0009_LTVF_TYPE,VFE_0005_VEHICLE_OPERATION_DIGIT1,VFE_0006_VEHICLE_OPERATION_DIGIT2,VFE_0007_CAB_HEIGHT,VFE_0008_HAS_PTO,LAST_RUN.1
0,2016-06-15,O-774932,16967,31.000000,159.000000,285.000000,2649.000000,9595.000000,4.263700e+04,3.157360e+05,2.950290e+05,4.182200e+04,10519.000000,68.000000,2450.000000,262.000000,255.000000,52.000000,15.000000,18.000000,57.000000,22.0,36.0,92.0,25.000000,70.0,19.0,34.000000,15.000000,36.00000,50.000000,77.000000,33.000000,9.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,BR-DILW,FAL7.5,2COMP900,4*2,TURBO-S,RAA11,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A1,DRC-AUTO,FSS-LEAF,UPTOENGR,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,AT261,RC-SMOOT,CTILTP-M,EML-BAS,TRACTOR,UEPTT,CABH215,UACTST,ADRC-FL,Low,Med,K,RAL13,RAL13,INTERREG_HAUL,2,8,CABH215,1,2019-05-14 10:32:00
1,2016-07-01,O-774932,16983,957.000000,3324.500000,19822.500000,65840.500000,208893.000000,7.008375e+05,3.023229e+06,2.872055e+06,6.577500e+05,211192.000000,74.000000,73004.000000,23843.500000,3344.500000,673.000000,296.000000,80.000000,62.000000,22.0,36.0,92.0,38.000000,70.0,19.0,41.000000,18.000000,44.50000,71.500000,149.500000,394.000000,32.179487,0.0,0.205128,0.205128,0.205128,0.0,0.0,...,BR-DILW,FAL7.5,2COMP900,4*2,TURBO-S,RAA11,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A1,DRC-AUTO,FSS-LEAF,UPTOENGR,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,AT261,RC-SMOOT,CTILTP-M,EML-BAS,TRACTOR,UEPTT,CABH215,UACTST,ADRC-FL,Low,Med,K,RAL13,RAL13,INTERREG_HAUL,2,8,CABH215,1,2019-05-14 10:32:00
2,2016-07-15,O-774932,16997,1552.903226,5433.080645,35088.822581,119192.016129,364449.483871,1.196003e+06,5.060461e+06,4.821555e+06,1.116311e+06,372215.258065,79.870968,133043.903226,41851.564516,5409.048387,1202.967742,487.709677,119.967742,68.322581,22.0,36.0,92.0,46.580645,70.0,19.0,46.870968,23.870968,48

In [ ]:
# Checking number of trucks in raw data

len((sorted_data['T_CHASSIS'].unique()))

6675

In [ ]:
data_repairs = pd.read_csv(Repairs_Edited_v1,low_memory=True)
repair_data_np = data_repairs.values
column_names_repair_data = data_repairs.columns
Chassis_numbers_in_repair_data = repair_data_np[:,0]


len((data_repairs['T_CHASSIS'].unique()))

In [ ]:
# Cross checking the raw dataset with repair dataset and keeping only the rows which have a chassis that is present in repairs 

temp = []
column_names_raw_data = sorted_data.columns.to_list()

for row in sorted_data.values:
  if row[1] in Chassis_numbers_in_repair_data:
    temp.append(row)

temp = np.array(temp)
sorted_data_with_only_repair_chassis = pd.DataFrame(temp, columns=column_names_raw_data)


sorted_data_with_only_repair_chassis['Malfunction_Label']= np.nan
sorted_data_with_only_repair_chassis['RUL']= np.nan
column_names_raw_data.append('Malfunction_Label')
column_names_raw_data.append('RUL')

In [ ]:
from datetime import datetime as dt
import time 
# sorted_data_with_only_repair_chassis = sorted_data_with_only_repair_chassis.drop(['Unnamed: 0'], axis=1) #Unneccessary column
Chassis_numbers_of_repairs_in_raw_data =  list(set(sorted_data_with_only_repair_chassis.values[:,1]))
sorted_data_with_only_repair_chassis_np = sorted_data_with_only_repair_chassis.values
column_names_repair_data = data_repairs.columns.to_list()


RepairDate_Index                              = column_names_repair_data.index('RepairDate')
Chassis_Index_in_repairs_dataset              = column_names_repair_data.index('T_CHASSIS')
Biweekly_Send_Date_Index                      = column_names_raw_data.index('BIWEEK_SEND_DATE')
RUL_Index                                     = column_names_raw_data.index('RUL')


for repair_row in repair_data_np:
  chassis = repair_row[Chassis_Index]

  if chassis in Chassis_numbers_of_repairs_in_raw_data:
    #print(repair_row)
    #print(chassis)


    repair_date = dt.strptime(repair_row[RepairDate_Index], "%Y-%m-%d %H:%M:%S.%f")
    #print ('repair data -',repair_date)
    Chassis_Index_in_raw_dataset = Chassis_numbers_of_repairs_in_raw_data.index(chassis)

    #start_ptr = start_end[Chassis_Index_in_raw_dataset][0] #0 th index is the start row of the particular chassis number in the csv
    #end_ptr = start_end[Chassis_Index_in_raw_dataset][1] #1 st index is the end row of the particular chassis number in the csv
    chassis_data = sorted_data_with_only_repair_chassis[sorted_data_with_only_repair_chassis['T_CHASSIS'] == chassis]
    start_ptr, end_ptr = chassis_data.index[0], chassis_data.index[-1]

    current_status_date  = dt.strptime(sorted_data_with_only_repair_chassis_np[start_ptr, Biweekly_Send_Date_Index], "%Y-%m-%d")
    #print ('chassis date -',current_status_date)

    # tracks if malfunction label got updated or not
    rul_break = False
    #print (start_ptr, end_ptr)
    while start_ptr <= end_ptr :
      current_status_date  = dt.strptime(sorted_data_with_only_repair_chassis_np[start_ptr, Biweekly_Send_Date_Index], "%Y-%m-%d")

      if repair_date >= current_status_date :
        #print ('---------',repair_date, current_status_date)
        rul = int((repair_date - current_status_date).days)
        #print( rul)
        sorted_data_with_only_repair_chassis_np[start_ptr,RUL_Index] =rul
        sorted_data_with_only_repair_chassis.loc[start_ptr, 'RUL'] = rul
        sorted_data_with_only_repair_chassis.loc[start_ptr, 'Malfunction_Label'] = 0
        start_ptr+=1
      else:
        sorted_data_with_only_repair_chassis.loc[start_ptr-1, 'Malfunction_Label'] = 1
        # break
        # no point in continuing
        rul_break = True
        break   

    # check if the malfunction value was updated
    if rul_break == False:
      sorted_data_with_only_repair_chassis.loc[start_ptr-1, 'Malfunction_Label'] = 1  
      

col_names_sorted_data = (sorted_data_with_only_repair_chassis.columns).tolist()

print(sorted_data_with_only_repair_chassis_np)



[['2016-05-01' 'N-767941' 16922 ... '2019-05-14 10:32:00' 0 39]
 ['2016-05-15' 'N-767941' 16936 ... '2019-05-14 10:32:00' 0 25]
 ['2016-06-01' 'N-767941' 16953 ... '2019-05-14 10:32:00' 1 8]
 ...
 ['2019-04-15' 'O-694033' 18001 ... '2019-05-14 10:32:00' 0 nan]
 ['2019-05-01' 'O-694033' 18017 ... '2019-05-14 10:32:00' 0 nan]
 ['2019-05-15' 'O-694033' 18031 ... '2019-05-14 10:32:00' 0 nan]]


In [ ]:
# save the new rul dataframe
sorted_with_rul = sorted_data_with_only_repair_chassis[(sorted_data_with_only_repair_chassis['RUL'] >= 0)]
sorted_with_rul.reset_index(drop='index', inplace=True)
sorted_with_rul.to_csv(Raw_Edited_v3, index=False, sep=',')

In [ ]:
sorted_data_with_only_repair_chassis.shape

(30483, 449)

In [ ]:
sorted_data_with_only_repair_chassis[(sorted_data_with_only_repair_chassis['T_CHASSIS'] == 'N-767941') & (sorted_data_with_only_repair_chassis['RUL'] >= 0)]

,BIWEEK_SEND_DATE,T_CHASSIS,INT_BWSD,LX_PMU_P1FH1_SLOPE_X_INDEX_10,LX_PMU_P1FH1_SLOPE_X_INDEX_11,LX_PMU_P1FH1_SLOPE_X_INDEX_12,LX_PMU_P1FH1_SLOPE_X_INDEX_13,LX_PMU_P1FH1_SLOPE_X_INDEX_14,LX_PMU_P1FH1_SLOPE_X_INDEX_15,LX_PMU_P1FH1_SLOPE_X_INDEX_16,LX_PMU_P1FH1_SLOPE_X_INDEX_17,LX_PMU_P1FH1_SLOPE_X_INDEX_18,LX_PMU_P1FH1_SLOPE_X_INDEX_19,LX_PMU_P1FH1_SLOPE_X_INDEX_2,LX_PMU_P1FH1_SLOPE_X_INDEX_20,LX_PMU_P1FH1_SLOPE_X_INDEX_21,LX_PMU_P1FH1_SLOPE_X_INDEX_22,LX_PMU_P1FH1_SLOPE_X_INDEX_23,LX_PMU_P1FH1_SLOPE_X_INDEX_24,LX_PMU_P1FH1_SLOPE_X_INDEX_25,LX_PMU_P1FH1_SLOPE_X_INDEX_26,LX_PMU_P1FH1_SLOPE_X_INDEX_27,LX_PMU_P1FH1_SLOPE_X_INDEX_28,LX_PMU_P1FH1_SLOPE_X_INDEX_29,LX_PMU_P1FH1_SLOPE_X_INDEX_3,LX_PMU_P1FH1_SLOPE_X_INDEX_30,LX_PMU_P1FH1_SLOPE_X_INDEX_31,LX_PMU_P1FH1_SLOPE_X_INDEX_4,LX_PMU_P1FH1_SLOPE_X_INDEX_5,LX_PMU_P1FH1_SLOPE_X_INDEX_6,LX_PMU_P1FH1_SLOPE_X_INDEX_7,LX_PMU_P1FH1_SLOPE_X_INDEX_8,LX_PMU_P1FH1_SLOPE_X_INDEX_9,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_1,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_10,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_11,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_12,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_13,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_14,LX_PMU_P1FWM_ENGINE_SPEED_TORQUE_H_X_INDEX_15,...,VAR_VWX_COMPRESSOR_CAPACITY,VAR_DAX_AXLE_ARRANGEMENT,VAR_H1B_TURBO_APPLICATION,VAR_FDX_REAR_AXLE_ARRANGEMENT,VAR_7PX_HYDRAULIC_PUMP_GEARBOX_MOUNTED,VAR_7NA_FRONT_SUSPENSION_INSTALLATION,VAR_LKX_DRIVER_STATE_SENSING_SYSTEM,VAR_QWX_TURBO,VAR_FZX_FRONT_AXLE_ARRANGEMENT,VAR_V6X_RETARDER,VAR_K4C_VEHICLE_UPTIME_SETTINGS,VAR_0RA_REGENERATION_CONTROLS_DPF,VAR_YKX_FRONT_SUSPENSION_SYSTEM,VAR_T3X_PTO_ENGINE_REAR_MOUNTED,VAR_PJX_ALTERNATOR_CAPACITY,VAR_41X_PARKING_HEATER,VAR_D7C_WHOLE_VEHICLE_TYPE_APPROVAL,VAR_5JB_TELEMATICS_COMMUNICAT_GATEWAYS,VAR_RWX_TRANSMISSION,VAR_DHX_ROAD_CONDITION,VAR_2EX_CAB_TILT_PUMP,VAR_D9C_EMISSION_LEVEL,VAR_DDX_CHASSIS_TYPE,VAR_O8A_ENGINE_PTO_TORQUE_CAPACITY,VAR_33A_CAB_HEIGHT,VAR_J2A_ACTIVE_STEERING,VAR_N9C_ADR_CLASSIFICATION,VFE_0001_HORSE_POWER,VFE_0002_GROSS_COMBINATION_WEIGHT_BIN,VFE_0003_ENGINE_GENERATION,VFE_0004_REAR_AXLE_LOAD_BIN,VFE_0005_REAR_AXLE_RATIO_BIN,VFE_0009_LTVF_TYPE,VFE_0005_VEHICLE_OPERATION_DIGIT1,VFE_0006_VEHICLE_OPERATION_DIGIT2,VFE_0007_CAB_HEIGHT,VFE_0008_HAS_PTO,LAST_RUN.1,Malfunction_Label,RUL
0,2016-05-01,N-767941,16922,315656.0,496675.0,969299.0,2030378.0,3743489.0,7958141.0,23250080.0,21877993.0,7834793.0,3716118.0,2255.0,2006061.0,1008061.0,621416.0,354023.0,302163.0,154487.0,113420.0,68134.0,31293.0,39278.0,14306.0,17461.0,4945.0,45373.0,32145.0,63026.0,122601.0,105194.0,237784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2COMP900,6*4,TURBO-D,RAA22,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A2,NONE,FSS-LEAF,PTOENG-R,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,ATO35,RC-ROUGH,CTILTP-E,EML-BAS,TRACTOR,EPTT650,CABH215,UACTST,UADRC,High,Med,K,HIGH,RAL26,HEAVY_CONSTR,1,2,CABH215,1,2019-05-14 10:32:00,0,39.0
1,2016-05-15,N-767941,16936,325041.5,511153.0,995783.5,2084542.5,3823374.0,8083753.5,23441404.0,22075118.5,7954288.5,3790668.5,2255.0,2060664.5,1034666.5,636365.0,362907.5,309492.5,159553.0,121808.5,72679.0,33810.5,42221.0,15110.5,18778.5,5035.5,49102.0,34277.0,68083.5,130569.5,108928.5,243480.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2COMP900,6*4,TURBO-D,RAA22,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A2,NONE,FSS-LEAF,PTOENG-R,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,ATO35,RC-ROUGH,CTILTP-E,EML-BAS,TRACTOR,EPTT650,CABH215,UACTST,UADRC,High,Med,K,HIGH,RAL26,HEAVY_CONSTR,1,2,CABH215,1,2019-05-14 10:32:00,0,25.0
2,2016-06-01,N-767941,16953,331225.5,519298.5,1012932.0,2119967.0,3881924.0,8220291.0,23812182.5,22413592.5,8077732.5,3848678.0,2276.0,2095507.0,1053592.0,646822.0,368683.0,313107.5,162794.5,125879.0,74443.5,34075.5,42584.0,15389.0,19223.0,5106.0,49504.0,34468.5,70037.5,133766.5,110717.5,247485.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2COMP900,6*4,TURBO-D,RAA22,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A2,NONE,FSS-LEAF,PTOENG-R,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,ATO35,RC-ROUGH,CT